In [1]:
import math
import datetime
from matplotlib import pyplot as plt
from FinMind.Data import Load
import numpy as np
import json
import pandas as pd
TaiwanStockInfo = Load.FinData(dataset = 'TaiwanStockInfo')

In [9]:
stock_id=2330
some_days_ago=700
rolling_num=20
nfactor=1
stop_loss_factor=1

data2 = Load.FinData(dataset = 'TaiwanStockPrice',select = [str(stock_id)],date = str(datetime.date.today()-datetime.timedelta(days = some_days_ago))) 
  

In [31]:
def get_critical_point(data,rolling_num):
    critical_point_data=pd.DataFrame()
    critical_point_data['close']=data['close']
    critical_point_data['rmax']=data['close'].rolling(rolling_num,min_periods=1,center=True).max()
    critical_point_data['rmin']=data['close'].rolling(rolling_num,center=True,min_periods=1).min()
    critical_point_data['critical']=np.logical_or(critical_point_data['close']==critical_point_data['rmax'],
                                                  critical_point_data['close']==critical_point_data['rmin'])
    return critical_point_data

In [46]:
def get_simple_wave(critical_point_data):
    critical_point_list=critical_point_data['critical']
    simple_wave       =critical_point_data[['close']][critical_point_list]
    simple_wave['index']= simple_wave.index
    simple_wave['gap']=simple_wave['close']-simple_wave['close'].shift(1)
    ans=map(lambda x:'up' if x>0  else 'down' if x<0 else 'flat' if x==0 else 'non',simple_wave['gap'])
    simple_wave['type']=list(ans)
    simple_wave=simple_wave[simple_wave['type']!='flat']
    simple_wave['next_type']=simple_wave['type'].shift(-1)
    simple_wave=simple_wave[simple_wave['next_type']!=simple_wave['type']]
    del simple_wave['next_type']
    return simple_wave
    

In [84]:
def get_wave_info(simple_wave,critical_point_data):
    wave_info=critical_point_data.iloc[simple_wave.index][['close']]
    wave_info['next_close']=wave_info['close'].shift(-1)
    wave_info['type']=simple_wave['type'].shift(-1)
    wave_info['mid_point']=(wave_info['close']+wave_info['close'].shift(-1))/2
    wave_info['last_mid_point']=wave_info['mid_point'].shift(1)
    wave_info['type1_buy']=np.logical_and(list(wave_info['type']=='down'),list(wave_info['next_close']>= wave_info['last_mid_point']))
    wave_info['type1_sell']=np.logical_and(list(wave_info['type']=='up'),list(wave_info['next_close']<wave_info['last_mid_point']))
    wave_info['index']=wave_info.index
    wave_info['next_index']=wave_info['index'].shift(-1)
    return wave_info

In [96]:
def test1(stock_id,some_days_ago=700,rolling_num=20,nfactor=1,stop_loss_factor=10):
    
    try:
        
        data                 = Load.FinData(dataset = 'TaiwanStockPrice',select = [str(stock_id)],date = str(datetime.date.today()-datetime.timedelta(days = some_days_ago)))#資料取得        
        critical_point_data  =  get_critical_point(data,rolling_num)            #簡化波型(粗糙)       
        simple_wave          =  get_simple_wave(critical_point_data)            #處理簡化過的波型 把兩個下跌的波整併成一個波        
        wave_info            =  get_wave_info(simple_wave,critical_point_data)  #計算使用波浪理論需要的參數        
        lose_list              = []
        earn_list              = []
        
        for i in wave_info[wave_info['type1_buy']].iterrows():
            start_index            = i[1]['next_index']         #起始日期的INDEX
            list_obj               = i[1]                       #上面那個表(essential_data)的一列
            now_index              = start_index                #開始迴圈時當前日期的index   
            start_point            = list_obj['close']          #第一波的最高點 當超過最高點時就啟動了
            wave_control_condition = list_obj['last_mid_point'] #第一波的中間點 當低於這個點波型破壞 
            first_wave_gap         = abs(start_point-wave_control_condition)*2
            purchase               = False
            purchase_price         = 0
            stop_loss_factor       = stop_loss_factor           #跌超過20%就停損
            nfactor                = nfactor                    #漲超過第一波的n倍就停利

            #logic compute
            while(now_index<=(len(data)-1)):
                now_price    =data['close'][now_index]
                now_index+=1
                if(purchase):
                            if -now_price+purchase_price>purchase_price*(1-stop_loss_factor/100):
                                purchase         =False
                                lose_list.append([-now_price+purchase_price,now_price,start_index,now_index,first_wave_gap])
                                break

                            if now_price<wave_control_condition:
                                losemoney=data['close'][start_index]-data['close'][now_index]
                                lose_list.append([losemoney,now_price,start_index,now_index,first_wave_gap])
                                purchase         =False
                                break
                            elif (now_price-purchase_price)>first_wave_gap*nfactor:
                                purchase         =False
                                earn_list.append([now_price-purchase_price,now_price,start_index,now_index,first_wave_gap])
                                break

                elif(not purchase):
                            if now_price<wave_control_condition:
                                break
                            elif now_price> start_point:
                                purchase       = True
                                purchase_price = now_price

        earn_list=pd.DataFrame(earn_list,columns=['賺了','當前價格','買入日','賣出日','波一漲幅'])
        earn_list['號碼']=stock_id
        lose_list=pd.DataFrame(lose_list,columns=['賠了','當前價格','買入日','賣出日','波一漲幅'])
        lose_list['號碼']=stock_id
        return [earn_list,lose_list]
    
    except KeyError:
        print('沒抓到資料')
        
   

In [92]:
stock_pocket=[2201,2204,2206,2207]
earn_list=pd.DataFrame()
lost_list=pd.DataFrame()
for stock_id in stock_pocket:
    print(stock_id)
    rs=test1(stock_id,700,nfactor=2,stop_loss_factor=20)
    if rs !=None:
        earn_list=pd.concat([earn_list,rs[0]])
        lost_list=pd.concat([lost_list,rs[1]])

2201
2204
2206
2207


In [94]:
lost_list

,賠了,當前價格,買入日,賣出日,波一漲幅,號碼
0,0.5,26.75,185.0,244.0,2.35,2204
0,5.0,345.50,32.0,80.0,14.50,2207


In [95]:
earn_list

,賺了,當前價格,買入日,賣出日,波一漲幅,號碼
0,4.05,23.95,408.0,451.0,1.4,2201
0,119.50,379.50,356.0,403.0,57.0,2207
